# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingenieria de Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 06**: Big Data Pipeline for Netflix data

**Fecha**:21/02/2025

**Nombre del Estudiante**:David Abraham Naranjo Salgado, Benjamin Leonardo Zarate Solano y Angel David Cortes Pacheco

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://873bad4e62fe:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/08 18:59:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Netflix service

In [4]:
from team_bellaco.spark_utils import SparkUtils

netflix_schema = SparkUtils.generate_schema([
    ("show_id", "string"), 
    ("type", "string"),
    ("title", "string"),
    ("director", "string"),
    ("country", "string"),
    ("date_added", "date"),
    ("release_year", "integer"),
    ("rating", "string"),
    ("duration", "string"),
    ("listed_in", "string")
])

show_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")


show_df.printSchema()

show_df.show(5, truncate=False)


root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead            |

### Limpiar datos

In [10]:
# Limpiar el DataFrame según los requisitos
clean_show_df = SparkUtils.clean_df(show_df)
# Mostrar el esquema limpio
clean_show_df.printSchema()

Filas originales: 8791, Filas después de limpieza: 0
root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)



In [9]:
clean_show_df.show(5, truncate=False)

+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+



### Escribri el esquema

# Escribir el DataFrame limpio particionado por release_year
SparkUtils.write_df(
    clean_show_df, 
    "/home/jovyan/notebooks/data/clean_netflix", 
    format="parquet",
    partition_by="release_year"  # Aquí especificamos la partición por año
)

In [ ]:
# Stop the SparkContext
sc.stop()